In [ ]:
import torch
import torch.nn as nn
from transformers_interpret import SequenceClassificationExplainer

class CustomBERTModel(nn.Module):
    def __init__(self):
        super(CustomBERTModel, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels = 77)

#         checkpoint = torch.load(chk_path, map_location=torch.device('cpu'))
        ### New layers:
#         self.linear1 = nn.Linear(768, 77)
        for self.param in self.bert.bert.parameters():
    #         print("HERE")
            self.param.requires_grad = False
        self.classifier = nn.Linear(77, 15) 

    def forward(self, input_ids, attention_mask, labels=None):
        output = torch.sigmoid(self.bert(input_ids, attention_mask=attention_mask)[0])
        #     print(output)
        output = self.classifier(output)
#         output = torch.sigmoid(output)
        #     print(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
import torch.nn as nn
import pytorch_lightning as pl 
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

from transformers import (
    BertForMaskedLM,
    AutoTokenizer,
    AutoConfig,
    BertModel, 
    BertConfig, 
    BertTokenizer, 
    BertForSequenceClassification
)
def print_model_info(model):
    # Get all of the model's parameters as a list of tuples.
    params = list(model.named_parameters())

    print('The BERT model has {:} different named parameters.\n'.format(len(params)))

    print('==== Embedding Layer ====\n')

    for p in params[0:5]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== First Transformer ====\n')

    for p in params[5:21]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== Output Layer ====\n')

    for p in params[-4:]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

def tokenize_sentences(sentences):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # encode_plus will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the [CLS] token to the start.
        #   (3) Append the [SEP] token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to max_length
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 320,           # Pad & truncate all sentences.
                            padding='max_length',
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids,attention_masks        

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

chk_path = "/intentidentification/multiclass/checkpoints_OOS/best-checkpoint.ckpt"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

checkpoint = torch.load(chk_path, map_location=torch.device('cpu'))

# config = BertConfig.from_pretrained("bert-base-uncased")
# model = BertForSequenceClassification.from_pretrained(
#     "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = len(LABEL_COLUMNS),
#     state_dict=checkpoint
# )

model = CustomBERTModel()

print_model_info(model)

#load specified model state
# print(checkpoint["state_dict"].get('bert.embeddings.word_embeddings.num_embeddings'))
model.load_state_dict(checkpoint["state_dict"])
model.eval()
model.to(device)

In [ ]:
LABEL_COLUMNS = ['replacement_card_duration',
 'expiration_date',
 'damaged_card',
 'improve_credit_score',
 'report_lost_card',
 'card_declined',
 'credit_limit_change',
 'apr',
 'redeem_rewards',
 'credit_limit',
 'rewards_balance',
 'application_status',
 'credit_score',
 'new_card',
 'international_fees']

In [ ]:
while(True):
    string = input("How may I help you? Question: ")
    #tokenize inputted sentence to be compatible with BERT inputs
    token_ids,attention_masks = tokenize_sentences([string])
    #get a tensor containing probabilities of inputted sentence being irrelevant or relevant
    model_outputs = (model(token_ids.to(device), attention_mask=attention_masks.to(device)))
#     print(model_outputs)
    softmax_layer = torch.nn.Softmax()
    result = softmax_layer(model_outputs[1])
#     print(result)
    #identify which output node has higher probability and what that probability is
    prediction = torch.argmax(result).item()
    confidence = torch.max(result).item()
    print("I see. The problem is related to: " + LABEL_COLUMNS[prediction].replace("_", ' ') + " with {:.2f}% confident".format(confidence*100))
    print("-"*80)

In [ ]:
multiclass_explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)

while(True):
    string = input("How may I help you? Question: ")
    #tokenize inputted sentence to be compatible with BERT inputs
    token_ids,attention_masks = tokenize_sentences([string])
    #get a tensor containing probabilities of inputted sentence being irrelevant or relevant
    model_outputs = (model(token_ids.to(device), token_type_ids=None, attention_mask=attention_masks.to(device)))
    softmax_layer = torch.nn.Softmax()
    result = softmax_layer(model_outputs[0])
    #identify which output node has higher probability and what that probability is
    prediction = torch.argmax(result).item()
    confidence = torch.max(result).item()
    print("I see. The problem is related to: " + LABEL_COLUMNS[prediction].replace("_", ' ') + " with {:.2f}% confident".format(confidence*100))
    word_attributions = multiclass_explainer(text=string)
    print("-"*80)
    print(word_attributions)
    html = multiclass_explainer.visualize()
    print(html)
    print("*"*100)
    print("*"*100)
    